MNIST 데이터로 배치로 숫자인식하기

In [ ]:
# coding: utf-8
import sys
sys.path.insert(0, r'C:\Users\USER\ms-deep-learning\deep-learning\notebooks\deep-learning-basic')
import numpy as np
import pickle
from dataset.mnist import load_mnist
from common.functions import sigmoid, softmax
import matplotlib.pyplot as plt

In [2]:
# 테스트(예측)해보기 위한 테스트 데이터 가져오는 함수 정의
def get_data():
    (X_train, t_train), (X_test, t_test) = load_mnist(normalize=True,
                                                      flatten= True,
                                                       one_hot_label=False)
    return X_test, t_test

# 학습된 신경망(모델) 가중치값 파일 로딩 
def init_network():
    with open("./sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    return network

In [3]:
# 예측하는 함수 정의
def predict(network, x):
    w1, w2, w3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    # 1번째 레이어
    a1 = np.dot(x, w1) + b1
    z1 = sigmoid(a1)

    # 2번째 레이어
    a2 = np.dot(z1, w2) + b2
    z2 = sigmoid(a2)

    # 3번째 레이어
    a3 = np.dot(z2, w3) + b3
    y = softmax(a3)

    return y

위에까지는 neuralnet_mnist.ipynb와 동일

입력데이터를 또 다른 행렬(매트릭스)로 묶어서(=배치) 처리! -> I/O 부하 줄일 수 있음 

In [15]:
# 테스트 데이터 가져오기
X, t = get_data()

# 학습된 신경망 가져오기
network = init_network()

# 정확도 측정을 위해 맞춘 개수 변수
accuracy_cnt = 0

### 배치 크기 지정 ###
batch_size = 100

for i in range(0, len(X), batch_size):
    # 배치 지정
    X_batch = X[i : i+batch_size] 
    # 테스트 데이터의 i번째 배치 신경망 출력 계산
    y_batch = predict(network, X_batch)  
    # 배치 출력 중 확률이 가장 높은 노드의 '인덱스' 반환 
    p = np.argmax(y_batch, axis=1)    
    # 계산해서 확률이 가장 높았던 노드의 인덱스와 정답(레이블) 비교 (단순하게 작성)
    accuracy_cnt += np.sum( p == t[i:i+batch_size]) # 맞았으면 정답 카운트 +1
    
    ### 틀린 데이터의 인덱스 찾기 ###
    #틀린 데이터의 인덱스 
    incorrect_indices = []
    correct = (p == t[i:i+batch_size])
    # correct가 False인 위치를 찾아서 전체 데이터셋에서의 인덱스로 변환
    incorrect_in_batch = np.where(correct == False)[0]  # 배치 내에서의 인덱스
    incorrect_global = i + incorrect_in_batch  # 전체 데이터셋에서의 인덱스
    incorrect_indices.extend(incorrect_global)

print('Accuracy: ' + str(float(accuracy_cnt) / len(X)))
print(f'틀린 데이터 개수: {len(incorrect_indices)}')
print(f'틀린 데이터 인덱스: {incorrect_indices}')

Accuracy: 0.9352
틀린 데이터 개수: 9
틀린 데이터 인덱스: [np.int64(9905), np.int64(9925), np.int64(9940), np.int64(9941), np.int64(9943), np.int64(9944), np.int64(9970), np.int64(9982), np.int64(9986)]


C:\Users\USER\AppData\Local\Temp\ipykernel_12800\3082864004.py:11: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  network = pickle.load(f)


`VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got 'align=0'. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  network = pickle.load(f)`       
=> 해당 모델 파일이 이전 버전 넘파이로 작성되서 경고가 뜨는 것! 작동엔 문제 없다고 함